In [2]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-09-22 13:32:30--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.009s  

2024-09-22 13:32:30 (117 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [4]:
# #all unique words
# import re
# words = words = sorted(list(set(re.split(r'(\s+)', text))))
# print(len(words))

In [5]:
# vocab_size = len(words)
# print(words[7])

In [5]:
#using standard tokenization
import re
def tokenize(text):
    return re.findall(r'\w+|[^\w\s]|\n', text, re.UNICODE)

tokens = tokenize(text)
words = sorted(list(set(tokens)))
vocab_size = len(words)
print(vocab_size)

13332


In [6]:
# print(words[100])
# print(tokens[4])
import torch

In [7]:
for i,word in enumerate(words[:4]):
  print(i,word)

0 

1 !
2 $
3 &


In [27]:
#creating mapping from words to integers
word_to_int = {word: i for i, word in enumerate(words)}
int_to_word = {i: word for i, word in enumerate(words)}

encode = lambda word: [word_to_int[w] for w in tokenize(word)]
decode = lambda l: ' '.join([int_to_word[i] for i in l])

print(encode("Before Before \n Before"))
print(decode(encode("Before Before \nBefore")))

[264, 264, 0, 264]
Before Before 
 Before


In [9]:

#aba sabai lai data lai encode and store in the tensor
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([302927]) torch.int64
tensor([  926,   488,     9,     0,   264, 12958,  9761,  3145,  6621,     5,
         7118,  8397, 11267,     7,     0,     0,   109,     9,     0,  2267,
            5, 11267,     7,     0,     0,   926,   488,     9,     0,  2788,
         3212,  3041, 10312, 10058, 12127,  5307, 11963, 12127,  6115,    11,
            0,     0,   109,     9,     0,  2003,     7, 10312,     7,     0,
            0,   926,   488,     9,     0,   926,     5, 13315,  7856,   421,
         1522,  7698,  4314,  5825, 12127, 11974,  9330,     7,     0,     0,
          109,     9,     0,  2660,  7856,     4, 11821,     5, 12958,  7856,
            4, 11821,     7,     0,     0,   926,   488,     9,     0,  1432,
        12670,  7798,  7238,     5,  3118, 12958,     4,  8131,  7087,  4766])


In [10]:
#spliting data into train and validation sets

n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [11]:
block_size = 8
train_data[:block_size+1]

tensor([  926,   488,     9,     0,   264, 12958,  9761,  3145,  6621])

In [12]:
x = train_data[:block_size+1]
y = train_data[1:block_size+2]
for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is {context} the target: {target}")

when input is tensor([926]) the target: 488
when input is tensor([926, 488]) the target: 9
when input is tensor([926, 488,   9]) the target: 0
when input is tensor([926, 488,   9,   0]) the target: 264
when input is tensor([926, 488,   9,   0, 264]) the target: 12958
when input is tensor([  926,   488,     9,     0,   264, 12958]) the target: 9761
when input is tensor([  926,   488,     9,     0,   264, 12958,  9761]) the target: 3145
when input is tensor([  926,   488,     9,     0,   264, 12958,  9761,  3145]) the target: 6621


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cuda


In [14]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y




xb,yb = get_batch('train')
xb = xb.to(device)
yb = yb.to(device)
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[12032, 13321,  8349,  6301,    10,  5763,  7485,     5],
        [ 1241,  7087,  3844,  7708, 13158,  3109,  7383,  3766],
        [    5,  9870, 13321,  3751,  3118, 13321,  3363,     0],
        [    0,  2682,  8664,  6172,    11,     0,     0,  1577]],
       device='cuda:0')
targets:
torch.Size([4, 8])
tensor([[13321,  8349,  6301,    10,  5763,  7485,     5,     0],
        [ 7087,  3844,  7708, 13158,  3109,  7383,  3766,     7],
        [ 9870, 13321,  3751,  3118, 13321,  3363,     0, 12127],
        [ 2682,  8664,  6172,    11,     0,     0,  1577,     9]],
       device='cuda:0')


In [15]:
print(xb) # input to the transformer

tensor([[12032, 13321,  8349,  6301,    10,  5763,  7485,     5],
        [ 1241,  7087,  3844,  7708, 13158,  3109,  7383,  3766],
        [    5,  9870, 13321,  3751,  3118, 13321,  3363,     0],
        [    0,  2682,  8664,  6172,    11,     0,     0,  1577]],
       device='cuda:0')


In [16]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    #yo tei embedding table hola, training ma train garne. ani yesle dinxa kun word paxi k aauxa vaera probability
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets = None):
    logits = self.token_embedding_table(idx) #(B,T,C)
    #yo every batches ko every channel ko lookup table add vayera aayo
    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)  #sarara 1 dimensional vayo
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)
    return logits, loss

  def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size).to(device)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

# Generate text, ensure idx is on the correct device
generated_idx = m.generate(idx=torch.zeros((1, 1), dtype=torch.long).to(device), max_new_tokens=10)
print(decode(generated_idx[0].tolist()))



torch.Size([32, 13332])
tensor(9.9853, device='cuda:0', grad_fn=<NllLossBackward0>)

 LARTIUS gallimaufry Droop meacock nill Somerville stitchery castaways designs virginity


In [17]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
if torch.cuda.is_available():
    print("CUDA is available. You are using GPU!")
else:
    print("CUDA is not available. You are using CPU.")

CUDA is available. You are using GPU!


In [ ]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')
    xb, yb = xb.to(device), yb.to(device)
    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if steps % 10 == 0:
      print(loss.item())

print(loss.item())


9.799545288085938
9.749726295471191
9.825566291809082
9.729238510131836
9.779780387878418
9.70313549041748
9.808412551879883
9.616798400878906
9.70821475982666
9.765565872192383
9.787276268005371


In [28]:
# Generate text, ensuring idx is on the correct device
generated_idx = m.generate(idx=torch.zeros((1, 1), dtype=torch.long).to(device), max_new_tokens=10)
print(decode(generated_idx[0].tolist()))


[0, 1894, 7783, 3179, 4161, 11043, 6754, 5254, 3887, 13093, 2208]


0


In [18]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [23]:
import torch
import re
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16  # how many independent sequences will we process in parallel?
block_size = 6   # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 128
n_head = 4
n_layer = 4
dropout = 0.0

torch.manual_seed(1337)

!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

import re
def tokenize(text):
    return re.findall(r'\w+|[^\w\s]|\n', text, re.UNICODE)

tokens = tokenize(text)
words = sorted(list(set(tokens)))
vocab_size = len(words)

word_to_int = {word: i for i, word in enumerate(words)}
int_to_word = {i: word for i in enumerate(words)}

encode = lambda word: [word_to_int[w] for w in tokenize(word)]
decode = lambda l: ' '.join([int_to_word[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long).to(device)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,), device=device)
    x = torch.stack([data[i:i+block_size] for i in ix]).to(device)
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]).to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters, device=device)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size, device=device)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

model = BigramLanguageModel()
model = model.to(device)

print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()




--2024-09-22 14:03:28--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.3’

input.txt.3         100%[===================>]   1.06M  --.-KB/s    in 0.008s  

2024-09-22 14:03:28 (129 MB/s) - ‘input.txt.3’ saved [1115394/1115394]

4.2189 M parameters
step 0: train loss 9.6709, val loss 9.6732
step 100: train loss 5.8783, val loss 5.7884
step 200: train loss 5.6209, val loss 5.6304
step 300: train loss 5.4121, val loss 5.4718
step 400: train loss 5.3516, val loss 5.4099
step 500: train loss 5.2430, val loss 5.3109
step 600: train loss 5.1868, val loss 5.2887
step 700: train loss 5.1035, val loss 5.2104
step 800: train loss

KeyError: 0

In [24]:
torch.save(model.state_dict(), 'bigram_language_model.pth')
from google.colab import files
files.download('bigram_language_model.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [33]:
generated_idx = m.generate(idx=torch.zeros((1, 1), dtype=torch.long).to(device), max_new_tokens=20)
print(generated_idx[0].tolist())

[0, 6553, 1684, 3679, 4368, 392, 1608, 11403, 7151, 12946, 4191, 9959, 3222, 5793, 8814, 3196, 4251, 12958, 8440, 5512, 2796]
